In [27]:
import openai
import pandas as pd
import json
import numpy as np
import re


In [2]:
with open('./secret.json') as config_file:
    config_data = json.load(config_file)
    api_key = config_data.get("api_key")

In [18]:
Raw_df=pd.read_csv("../Data/table_A.csv")
Target_df=pd.read_csv("../Data/template.csv")
Raw_df_2=pd.read_csv("../Data/table_B.csv")

In [24]:
def extract_function_signature(text):
    pattern = r'def (\w+)\(([^)]+)\):'
    match = re.search(pattern, text)
    
    if match:
        function_name = match.group(1)
        variables = match.group(2)
        return f"Function Name: {function_name}, Variables: {variables}"
    else:
        return "Function signature not found"

In [32]:
def _Open_AI_ETL_Generator_(Raw_df:pd.DataFrame,Target_df:pd.DataFrame):
    #### douplicat the excat same columns to reduce number tokens. 
    temp_df=Raw_df[:5].T
    temp_df=temp_df.drop_duplicates().T
    Columns=temp_df.columns.to_list()
    #temp_df[Columns]
    Raw_df=Raw_df[Columns]
    ##### Instead of passing the entire target values, we send the sample to reduce the number of tokens. 
    ##### list(Target_df[:3].values


    # Specify the prompt for GPT-3.5
    prompt = f""" i have the following data,  data:\n{Raw_df.values.tolist()}\n\n 
    and I want a python function that can remove the unwanted and doplicated columns and select the columns that are similar to the traget_data ,
    target_data:\n{Target_df[:3].values.tolist()}\n\n . 
    data in the selected columns may have extra characters, please remove them and format the data the same as target_data.
    just retun the function ,
    """

    # Send a completion request to GPT-3.5
    response = openai.Completion.create(
        engine="text-davinci-003",  # You can specify the engine you prefer
        #engine="curie",
        prompt=prompt,
        max_tokens=3000,  # Adjust this based on the expected response length
        api_key=api_key
    )

    # Extract the suggestions and reformatted dataset from the response
    #suggestions_and_reformatted_data = response.choices[0].text
    return response

In [33]:
response=_Open_AI_ETL_Generator_(Raw_df=Raw_df,Target_df=Target_df)

In [34]:
print(response.choices[0].text)


def transformData(data):
    target_data = [['01-05-2023', 'John Doe', 'Gold', 'AB12345', 150], ['02-05-2023', 'Jane Smith', 'Silver', 'CD67890', 100], ['03-05-2023', 'Michael Brown', 'Bronze', 'EF10111', 50]]
    
    result = []
    for d in data:
        date_str = d[0]
        result_date = date_str[6:10] + '-' + date_str[3:5] + '-' + date_str[0:2]
        name = d[1]
        plan = d[2].split(" ")[0]
        id = d[3].split("-")[1]
        price = d[4]
        new_row = [result_date, name, plan, id, price]
        result.append(new_row)
        
    return result


In [35]:
Functtion_Name= extract_function_signature(response.choices[0].text)
Functtion_Name

'Function Name: transformData, Variables: data'

In [37]:
exec(response.choices[0].text)

In [38]:
transformData( Raw_df.values.tolist())

[['2023-01-05', 'John Doe', 'Gold', '12345', 150.0],
 ['2023-02-05', 'Jane Smith', 'Silver', '67890', 100.0],
 ['2023-03-05', 'Michael Brown', 'Bronze', '10111', 50.0],
 ['2023-04-05', 'Alice Johnson', 'Gold', '12121', 150.0],
 ['2023-05-05', 'Bob Wilson', 'Silver', '13131', 100.0],
 ['2023-06-05', 'Carol Martinez', 'Bronze', '14141', 50.0],
 ['2023-07-05', 'David Anderson', 'Gold', '15151', 150.0],
 ['2023-08-05', 'Eva Thomas', 'Silver', '16161', 100.0],
 ['2023-09-05', 'Frank Jackson', 'Bronze', '17171', 50.0],
 ['2023-10-05', 'Grace White', 'Gold', '18181', 150.0]]